In [1]:
import folium
import zipfile
import io

from fastkml import kml

In [2]:
# read KML content 
kmz = zipfile.ZipFile("qfaults.kmz", 'r')  # extract zip file first, then read kmz file inside the extracted folder
kml_content = kmz.open('doc.kml', 'r').read()  # kml content
k = kml.KML()
k.from_string(kml_content)

In [4]:
docs = list(k.features())
folders=[]
for d in docs:
    folders.extend(list(d.features()))
records = []
for f in folders:
    records.extend(list(f.features()))
geoms = [element.geometry for element in records]

In [5]:
dir(records[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_address',
 '_atom_author',
 '_atom_link',
 '_geometry',
 '_phoneNumber',
 '_snippet',
 '_styleUrl',
 '_styles',
 '_time_span',
 '_time_stamp',
 'address',
 'append_style',
 'author',
 'begin',
 'description',
 'end',
 'etree_element',
 'extended_data',
 'from_element',
 'from_string',
 'geometry',
 'id',
 'isopen',
 'link',
 'name',
 'ns',
 'phoneNumber',
 'snippet',
 'styleUrl',
 'styles',
 'targetId',
 'timeStamp',
 'to_string',
 'visibility']

In [64]:
# print(records[0].name)
# print(records[0].description)
# print(records[0].id)
print(records[0]._time_stamp)
dir(records[0]._styleUrl)

None


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'etree_element',
 'from_element',
 'from_string',
 'id',
 'ns',
 'targetId',
 'to_string',
 'url']

In [20]:
names = [element.name for element in records]
print(len(set(names)))
print(len(name))

2107
112527


In [ ]:
geoms = [element.geometry for element in records]  # extract geometry

In [24]:
dir(geoms[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__geo_interface__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_coordinates',
 '_geoms',
 '_type',
 'bounds',
 'geom_type',
 'geoms',
 'to_wkt',
 'wkt']

In [35]:
geoms[0].wkt

'MULTILINESTRING((-124.090694000324 40.11756400000441 0.0, -124.0905649995543 40.11569200027037 0.0, -124.0904559997744 40.11512000009119 0.0, -124.0901240004117 40.11466500032259 0.0, -124.089893000433 40.11393100124901 0.0, -124.0898769996411 40.1128669998052 0.0, -124.0898520004251 40.1112709997484 0.0, -124.0901279997113 40.10903199974997 0.0, -124.0894950000497 40.1031160001442 0.0, -124.0887710001515 40.10135000029333 0.0, -124.0873430004466 40.09905999987495 0.0, -124.0845850000479 40.09323699970534 0.0, -124.0819189997106 40.08657599997652 0.0, -124.0716099999854 40.06145300125832 0.0))'

In [33]:
data = [item.geom_type for item in geoms]

In [34]:
set(data)

{'MultiLineString'}